# Neuronales Netz zur Vorhersage von Fahrtzeiten

Importiert die benötigten Module und ließt csv ein

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from tensorflow import keras
import time
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.preprocessing import MinMaxScaler

train_x = pd.read_csv("../data/taxi_data/yellow_tripdata_2020-04.csv")
train_x

Presproccessing der eingelsenen csv

In [175]:
train_x['tpep_pickup_datetime'] = pd.to_datetime(train_x['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
train_x['tpep_dropoff_datetime'] = pd.to_datetime(train_x['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
train_x['trip_duration'] = (train_x['tpep_dropoff_datetime'] - train_x['tpep_pickup_datetime']).dt.seconds
train_x['PULocationID'].fillna(-1, inplace=True)
train_x['DOLocationID'].fillna(-1, inplace=True)
train_x['tpep_pickup_month'] = train_x['tpep_pickup_datetime'].dt.month
train_x['tpep_dropoff_month'] = train_x['tpep_dropoff_datetime'].dt.month
train_x['tpep_pickup_day_numeric'] = train_x['tpep_pickup_datetime'].dt.day
train_x['tpep_dropoff_day_numeric'] = train_x['tpep_dropoff_datetime'].dt.day
train_x['tpep_pickup_day_name'] = train_x['tpep_pickup_datetime'].dt.day_name()
train_x['tpep_dropoff_day_name'] = train_x['tpep_dropoff_datetime'].dt.day_name()
train_x['tpep_pickup_hour'] = train_x['tpep_pickup_datetime'].dt.hour
train_x['tpep_dropoff_hour'] = train_x['tpep_dropoff_datetime'].dt.hour
train_x['tpep_pickup_day'] = train_x['tpep_pickup_datetime'].dt.strftime("%w").astype(int)
train_x['tpep_dropoff_day'] = train_x['tpep_dropoff_datetime'].dt.strftime("%w").astype(int)
train_y = (train_x['tpep_dropoff_datetime'] - train_x['tpep_pickup_datetime']).dt.seconds
train_x['tpep_pickup_datetime'] = train_x['tpep_pickup_datetime'].apply(lambda x: int(x.strftime('%Y%m%d')))
train_x['tpep_dropoff_datetime'] = train_x['tpep_dropoff_datetime'].apply(lambda x: int(x.strftime('%Y%m%d')))
train_x = train_x.fillna(0)
train_x = train_x.drop(["tpep_dropoff_day_name"], axis=1)
train_x = train_x.drop(["tpep_pickup_day_name"], axis=1)
train_x = train_x.drop(["store_and_fwd_flag"], axis=1)

print(train_x)

        VendorID  tpep_pickup_datetime  tpep_dropoff_datetime  \
0              1              20200401               20200401   
1              1              20200401               20200401   
2              1              20200401               20200401   
3              1              20200401               20200401   
4              2              20200401               20200401   
...          ...                   ...                    ...   
238068         2              20200430               20200430   
238069         2              20200430               20200430   
238070         2              20200430               20200430   
238071         2              20200430               20200501   
238072         2              20200430               20200430   

        passenger_count  trip_distance  RatecodeID  PULocationID  \
0                   1.0           1.20         1.0            41   
1                   1.0           3.40         1.0            95   
2              

Normalisierung der Daten, sowie Teilung in Labels- und Traingsdaten

In [176]:
columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_month', 'tpep_pickup_day', 'tpep_pickup_hour', 'trip_duration']
# define min max scaler
train_data_scaler = MinMaxScaler()
train_x[columns] = train_data_scaler.fit_transform(train_x[columns])

y_train_labels = train_x["trip_duration"].values
x_train_samples = train_x[columns]
x_train_samples = x_train_samples.drop(["trip_duration"], axis=1)

print(x_train_samples)
print("---")
print(y_train_labels)



        PULocationID  DOLocationID  tpep_pickup_month  tpep_pickup_day  \
0           0.151515      0.087121                1.0         0.500000   
1           0.356061      0.742424                1.0         0.500000   
2           0.893939      0.515152                1.0         0.500000   
3           0.253788      0.534091                1.0         0.500000   
4           0.992424      0.276515                1.0         0.500000   
...              ...           ...                ...              ...   
238068      0.897727      0.280303                1.0         0.666667   
238069      0.992424      0.867424                1.0         0.666667   
238070      0.515152      0.155303                1.0         0.666667   
238071      0.106061      0.257576                1.0         0.666667   
238072      0.049242      0.106061                1.0         0.666667   

        tpep_pickup_hour  
0                    0.0  
1                    0.0  
2                    0.0  
3  

Konvertieren der Dataframes in Arrays

In [177]:
x_train_samples = np.array(x_train_samples)
y_train_labels = np.array(y_train_labels)
print(y_train_labels)

[0.01424785 0.00931724 0.0062385  ... 0.00902788 0.02633132 0.00996539]


Erstellen eines Modells für Regression

In [178]:
model = Sequential([
    Dense(units=32, input_shape=(5, 1), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=32, activation='relu'),
    Flatten(),
    Dense(units=1)
])
model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_squared_error'])
model.fit(x=x_train_samples, y=y_train_labels, validation_split=0.1, batch_size=1000, epochs=10, shuffle=True,
          verbose=2)




Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 5, 32)             64        
                                                                 
 dense_53 (Dense)            (None, 5, 32)             1056      
                                                                 
 dense_54 (Dense)            (None, 5, 32)             1056      
                                                                 
 flatten_5 (Flatten)         (None, 160)               0         
                                                                 
 dense_55 (Dense)            (None, 1)                 161       
                                                                 
Total params: 2,337
Trainable params: 2,337
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
215/215 - 1s - loss: 9.9721e-04 - mean_s

Einlesen einer weiteren csv für den Folgemonat um vorhersagen machen zu können. Zudem preprocessing der Daten


In [214]:
test_data = pd.read_csv("../data/taxi_data/yellow_tripdata_2020-04.csv", nrows=3)
test_data['tpep_pickup_datetime'] = pd.to_datetime(test_data['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
test_data['tpep_dropoff_datetime'] = pd.to_datetime(test_data['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
test_data['trip_duration'] = (test_data['tpep_dropoff_datetime'] - test_data['tpep_pickup_datetime']).dt.seconds
test_data['PULocationID'].fillna(-1, inplace=True)
test_data['DOLocationID'].fillna(-1, inplace=True)
test_data['tpep_pickup_month'] = test_data['tpep_pickup_datetime'].dt.month
test_data['tpep_dropoff_month'] = test_data['tpep_dropoff_datetime'].dt.month
test_data['tpep_pickup_day_numeric'] = test_data['tpep_pickup_datetime'].dt.day
test_data['tpep_dropoff_day_numeric'] = test_data['tpep_dropoff_datetime'].dt.day
test_data['tpep_pickup_day_name'] = test_data['tpep_pickup_datetime'].dt.day_name()
test_data['tpep_dropoff_day_name'] = test_data['tpep_dropoff_datetime'].dt.day_name()
test_data['tpep_pickup_hour'] = test_data['tpep_pickup_datetime'].dt.hour
test_data['tpep_dropoff_hour'] = test_data['tpep_dropoff_datetime'].dt.hour
test_data['tpep_pickup_day'] = test_data['tpep_pickup_datetime'].dt.strftime("%w").astype(int)
test_data['tpep_dropoff_day'] = test_data['tpep_dropoff_datetime'].dt.strftime("%w").astype(int)
test_data['tpep_pickup_datetime'] = test_data['tpep_pickup_datetime'].apply(lambda x: int(x.strftime('%Y%m%d')))
test_data['tpep_dropoff_datetime'] = test_data['tpep_dropoff_datetime'].apply(lambda x: int(x.strftime('%Y%m%d')))
test_data = test_data.fillna(0)
test_data = test_data.drop(["tpep_dropoff_day_name"], axis=1)
test_data = test_data.drop(["tpep_pickup_day_name"], axis=1)
test_data = test_data.drop(["store_and_fwd_flag"], axis=1)




Normalisierung der Testdaten 

In [215]:
columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_month', 'tpep_pickup_day', 'tpep_pickup_hour', 'trip_duration']
#make decisons based on csv input files
real_duration = test_data['trip_duration']
test_data[columns] = train_data_scaler.fit_transform(test_data[columns])

test_labels = test_data["trip_duration"].values
test_samples = test_data.drop(["trip_duration"], axis=1)

y_train_labels = train_x["trip_duration"].values
test_data_samples = test_data[columns]
test_data_samples = test_data_samples.drop(["trip_duration"], axis=1)


#make single custom predictions
# data = {'PULocationID': [95], 'DOLocationID': [197], 'tpep_pickup_month': [5], 'tpep_pickup_day': [3],
#         'tpep_pickup_hour': [0], 'trip_duration': [805]}
# custom_test_data = pd.DataFrame(data=data)
# real_duration = custom_test_data['trip_duration']
#
# print(custom_test_data)
# custom_test_data[columns] = train_data_scaler.fit_transform(custom_test_data[columns])
# test_labels = custom_test_data["trip_duration"].values
# test_data_samples = custom_test_data[columns]
# test_data_samples = test_data_samples.drop(["trip_duration"], axis=1)


In [216]:
predictions = model.predict(test_data_samples)

(3, 5)
1/1 [==============================] - 0s 19ms/step
(3, 1)


In [217]:
test_data_samples['trip_duration'] = predictions.flatten()
test_data_inversed = train_data_scaler.inverse_transform(test_data_samples)
column_values = ['PULocationID', 'DOLocationID', 'tpep_pickup_month', 'tpep_pickup_day', 'tpep_pickup_hour',
                 'predicted_trip_duration']

result = pd.DataFrame(data=test_data_inversed, columns=column_values)
result['real_duration'] = real_duration
print(result)


   PULocationID  DOLocationID  tpep_pickup_month  tpep_pickup_day  \
0          41.0          24.0                4.0              3.0   
1          95.0         197.0                4.0              3.0   
2         237.0         137.0                4.0              3.0   

   tpep_pickup_hour  predicted_trip_duration  real_duration  
0               0.0               551.972751           1231  
1               0.0               548.130482            805  
2               0.0               547.354790            539  


In [197]:
print(test_data_inversed[5])
print(real_duration)

plt.plot(real_duration, color='red', label='Real data')
plt.plot(test_data_inversed[5], color='blue', label='Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()

IndexError: index 5 is out of bounds for axis 0 with size 2